In [1]:
# common imports

import sys
sys.path.append("../datasets/ARID_supporting_scripts")


import os
import random
import mapper
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

seed = 15
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

2025-02-27 01:15:41.918254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 01:15:41.918295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 01:15:41.919600: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 01:15:41.926289: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-27 01:15:42.548054: W tensorflow/compiler/tf2

In [2]:
import datasets

dataset = datasets.load_from_disk('./ARID/')

In [3]:
dataset_train = dataset['train']
dataset_test = dataset['test']

In [4]:
# Injects random noise into the 'label' by flipping field in a balanced manner.

import random

def add_balanced_label_noise(dataset, noise_rate = 0.2, seed = seed):
    random.seed(seed)
    labels = dataset["signal_keyword"]
    unique_labels = set(labels)
    new_labels = list(labels)
    indices_by_class = {label: [] for label in unique_labels}
    for idx, label in enumerate(labels):
        indices_by_class[label].append(idx)
    for label, indices in indices_by_class.items():
        num_samples = len(indices)
        num_noisy = min(int(noise_rate * num_samples), num_samples - 1)
        noisy_indices = random.sample(indices, num_noisy)
        for idx in noisy_indices:
            possible_labels = list(unique_labels - {label})
            if possible_labels: 
                new_label = random.choice(possible_labels)
                new_labels[idx] = new_label
    return dataset.add_column("noisy_signal_keyword", new_labels)

In [5]:
noise_rate = 0.3
noisy_train_dataset = add_balanced_label_noise(dataset['train'], noise_rate = noise_rate)
noisy_train_dataset = noisy_train_dataset.remove_columns('label')

In [6]:
lbl_ = dataset['test'].features['label'].names
label2id = {lbl: idx for idx, lbl in enumerate(lbl_)}
id2label = {val: key for key, val in label2id.items()}

In [7]:
noisy_train_dataset = noisy_train_dataset.map(lambda x: {"label": label2id[x["noisy_signal_keyword"]]})
noisy_train_dataset

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Dataset({
    features: ['REQID', 'REQID_expanded', 'Requirement Sentences', 'Open/ Closed Source', 'class', 'signal_keyword', 'Source', 'noisy_signal_keyword', 'label'],
    num_rows: 1916
})

In [8]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


batch_size = 16


model_ckpt = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                        num_labels = len(lbl_),
                                                        id2label = id2label,
                                                        label2id = label2id,)

2025-02-27 01:15:48.020201: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-27 01:15:48.022219: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-27 01:15:48.026028: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
def preprocess_function(dataset):
    return tokenizer(dataset['Requirement Sentences'], truncation = True)

In [11]:
X_train_encoded = noisy_train_dataset.map(preprocess_function, batched = True)
X_test_encoded = dataset_test.map(preprocess_function, batched = True)

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

In [12]:
print(X_train_encoded['Requirement Sentences'][0])
print(X_train_encoded['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(X_train_encoded['input_ids'][0]))

The DWA must request DWA acknowledgment flashing when the DWA has assumed the "armed" state and the outer skin is closed.
[0, 133, 211, 8460, 531, 2069, 211, 8460, 38169, 22643, 77, 5, 211, 8460, 34, 9159, 5, 22, 17651, 113, 194, 8, 5, 15705, 3024, 16, 1367, 4, 2]
['<s>', 'The', 'ĠD', 'WA', 'Ġmust', 'Ġrequest', 'ĠD', 'WA', 'Ġacknowledgment', 'Ġflashing', 'Ġwhen', 'Ġthe', 'ĠD', 'WA', 'Ġhas', 'Ġassumed', 'Ġthe', 'Ġ"', 'armed', '"', 'Ġstate', 'Ġand', 'Ġthe', 'Ġouter', 'Ġskin', 'Ġis', 'Ġclosed', '.', '</s>']


In [13]:
X_train_encoded

Dataset({
    features: ['REQID', 'REQID_expanded', 'Requirement Sentences', 'Open/ Closed Source', 'class', 'signal_keyword', 'Source', 'noisy_signal_keyword', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1916
})

In [14]:
tf_train_dataset = model.prepare_tf_dataset(
    X_train_encoded,
    shuffle = True,
    batch_size = batch_size,
    tokenizer = tokenizer
)

tf_valid_dataset = model.prepare_tf_dataset(
    X_test_encoded,
    shuffle = False,
    batch_size = batch_size,
    tokenizer = tokenizer
)

In [15]:
from transformers import create_optimizer

num_epochs = 10
batches_per_epoch = len(X_train_encoded) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr = 2e-5, num_warmup_steps = 0, num_train_steps = total_train_steps
)

2025-02-27 01:15:51.940166: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [16]:
import evaluate
from transformers.keras_callbacks import KerasMetricCallback


def compute_metrics(eval_predictions):
    metric1 = evaluate.load("precision")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("f1")


    predictions, labels = eval_predictions
    predictions = np.argmax(predictions, axis = 1)

    precision = metric1.compute(predictions = predictions, references = labels, average = 'macro')["precision"]
    recall = metric2.compute(predictions = predictions, references = labels, average = 'macro')["recall"]
    f1 = metric3.compute(predictions = predictions, references = labels, average = 'macro')["f1"]
    return {"precision": precision, "recall": recall, "f1": f1}

metric_callback = KerasMetricCallback(metric_fn = compute_metrics, eval_dataset = tf_valid_dataset)

In [17]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

model_name = model_ckpt.split("/")[-1]
push_to_hub_model_id = f'{model_name}_noise_rate{noise_rate}_seed{seed}_percent_noise'
print(push_to_hub_model_id)
tensorboard_callback = TensorBoard(log_dir = f'./{push_to_hub_model_id}/logs')

push_to_hub_callback = PushToHubCallback(
    output_dir = f"./{push_to_hub_model_id}",
    tokenizer = tokenizer,
    hub_model_id = push_to_hub_model_id,
)

callbacks = [push_to_hub_callback, tensorboard_callback, metric_callback]

roberta-base_noise_rate0.3_seed15_percent_noise


/home/kasra/metal-engine/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/kasrahabib/roberta-base_noise_rate0.3_seed15_percent_noise into local empty directory.


In [18]:
model.compile(optimizer = optimizer)
history = model.fit(tf_train_dataset, validation_data = (tf_valid_dataset), epochs = num_epochs, callbacks = callbacks)

Epoch 1/10
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2025-02-27 01:16:34.082567: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ba5208560f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-27 01:16:34.082590: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-27 01:16:34.082596: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-27 01:16:34.087552: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-27 01:16:34.109803: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1740615394.167287 4042119 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


119/119 [==============================] - ETA: 0s - loss: 2.6751

Using the latest cached version of the module from /home/kasra/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--precision/4e7f439a346715f68500ce6f2be82bf3272abd3f20bdafd203a2c4f85b61dd5f (last modified on Mon May  6 14:56:12 2024) since it couldn't be found locally at evaluate-metric--precision, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/kasra/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--recall/e40e6e98d18ff3f210f4d0b26fa721bfaa80704b1fdf890fa551cfabf94fc185 (last modified on Mon May  6 14:56:14 2024) since it couldn't be found locally at evaluate-metric--recall, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/kasra/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/0ca73f6cf92ef5a268320c697f7b940d1030f8471714bffdb6856c641b818974 (last modified on Mon May  6 14:56:15 2024) since it couldn't be found locally at evaluate-metric-

119/119 [==============================] - 98s 438ms/step - loss: 2.6751 - val_loss: 1.8436 - precision: 0.3723 - recall: 0.3997 - f1: 0.3170
Epoch 2/10
  2/119 [..............................] - ETA: 8s - loss: 2.1525

...

119/119 [==============================] - 47s 394ms/step - loss: 0.9842 - val_loss: 0.8081 - precision: 0.7767 - recall: 0.7614 - f1: 0.7661
Epoch 10/10
119/119 [==============================] - ETA: 0s - loss: 0.9213

Several commits (10) will be pushed upstream.
Using the latest cached version of the module from /home/kasra/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--precision/4e7f439a346715f68500ce6f2be82bf3272abd3f20bdafd203a2c4f85b61dd5f (last modified on Mon May  6 14:56:12 2024) since it couldn't be found locally at evaluate-metric--precision, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/kasra/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--recall/e40e6e98d18ff3f210f4d0b26fa721bfaa80704b1fdf890fa551cfabf94fc185 (last modified on Mon May  6 14:56:14 2024) since it couldn't be found locally at evaluate-metric--recall, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/kasra/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/0ca73f6cf92ef5a268320c697f7b940d1030f8471714bffdb6856c641b818974 (last modified on Mon May  6 14:56:15 2024) since it

119/119 [==============================] - 46s 387ms/step - loss: 0.9213 - val_loss: 0.8176 - precision: 0.7736 - recall: 0.7604 - f1: 0.7637


Several commits (11) will be pushed upstream.
The progress bars may be unreliable.
EOF
error: failed to push some refs to 'https://huggingface.co/kasrahabib/roberta-base_noise_rate0.3_seed15_percent_noise'



OSError: EOF
error: failed to push some refs to 'https://huggingface.co/kasrahabib/roberta-base_noise_rate0.3_seed15_percent_noise'


In [19]:
y_pred = model.predict(tf_valid_dataset).logits

30/30 [==============================] - 6s 48ms/step


In [20]:
import numpy as np

y_pred = np.argmax(y_pred, axis = 1)
y_true = dataset['test']['label']

In [22]:
y_true_label = [id2label[i] for i in y_true]
y_pred_label = [id2label[i] for i in y_pred]

y_true_three_class = [mapper.map_hf[i] for i in y_true_label]
y_pred_three_class = [mapper.map_hf[i] for i in y_pred_label]

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(y_true, y_pred, average = 'binary'):
    print('Precision: ', precision_score(y_true, y_pred, average = average))
    print('Recall: ', recall_score(y_true, y_pred, average = average))
    print('f1_score: ', f1_score(y_true, y_pred, average = average))

In [24]:
# seed 15
evaluate(y_true_three_class, y_pred_three_class, average = 'macro')              

Precision:  0.906758564436907
Recall:  0.9082539682539683
f1_score:  0.9074034005854187


In [24]:
# seed 13
evaluate(y_true_three_class, y_pred_three_class, average = 'macro')              

Precision:  0.8912275071381522
Recall:  0.8910317460317461
f1_score:  0.8908946498700914


In [24]:
# seed 19
evaluate(y_true_three_class, y_pred_three_class, average = 'macro')              

Precision:  0.8885427483558325
Recall:  0.8904761904761905
f1_score:  0.8889673509617334


In [24]:
# seed 100
evaluate(y_true_three_class, y_pred_three_class, average = 'macro')              

Precision:  0.8863507639154383
Recall:  0.8847619047619047
f1_score:  0.8855219792111105


In [24]:
# seed 42
evaluate(y_true_three_class, y_pred_three_class, average = 'macro')              

Precision:  0.8622377622377622
Recall:  0.8610317460317461
f1_score:  0.8596212624584719
